在这篇博客中，我们将会应用seq2seq模型来尝试解决kaggle的一个竞赛，以下为竞赛地址：
https://www.kaggle.com/c/text-normalization-challenge-english-language

In [1]:
import numpy as np
import tensorflow as tf

# 重置默认计算图
tf.reset_default_graph()
# 创建交互式计算会话
sess = tf.InteractiveSession()

PAD = 0
EOS = 1

env = 'dev'  # 或者是 'prod'
mode = tf.contrib.learn.ModeKeys.TRAIN  # tf.contrib.learn.ModeKeys.INFER, tf.contrib.learn.ModeKeys.EVAL

在dev模式下创建tensorflow constant作为训练数据。时间步数最大为4，训练批次大小为三个时间序列，分别为：[2， 2， 3， 0]， [4， 5， 6， 7]， [8， 9， 0， 0]。对于序列1和3，我们在末尾补0.

In [6]:
if env == 'dev':
    # 创建时间序列
    # Batch Size为3
    # 时间步数为 4
    x = [[2, 4, 8],
         [2, 5, 9],
         [3, 6, 0],
         [0, 7, 0]]
    x_len = [3, 4, 2]

    encoder_inputs = tf.constant(x, dtype=tf.int32, name='encoder_inputs')
    encoder_inputs_length = tf.constant(x_len, dtype=tf.int32, name='encoder_inputs_length')
elif env == 'prod':
    raise NotImplementedError("创建placeholder")

创建encoder，设定输入词典大小为10，包括 PAD=0， EOS=1. 设定embedding_size为5

In [7]:
if env == 'dev':
    vocab_size = 10
    embedding_size = 5
    input_hidden_unit = 20
elif env == 'prod':
    raise NotImplementedError("设定数据集的vocab size， embedding size， input hidden size")

这里我们将会创建输入RNN网络，我们使用LSTM单元：
    <img src="resources/images/note_1/lstm.jpg">

In [ ]:
import math

with tf.variable_scope("embedding") as scope:
    encoder_cell = tf.contrib.rnn.LSTMCell(input_hidden_unit)
    sqrt3 = math.sqrt(3)
    initializer = tf.random_uniform_initializer(-sqrt3, sqrt3)

    embedding_matrix = tf.get_variable(
        name="embedding_matrix",
        shape=[vocab_size, embedding_size],
        initializer=initializer,
        dtype=tf.float32)

    embedded_encoder_inputs = tf.nn.embedding_lookup(
        embedding_matrix, encoder_inputs)
    
    outputs, state = tf.nn.dynamic_rnn(
        cell=encoder_cell,
        inputs=embedded_encoder_inputs,
        sequence_length=encoder_inputs_length,
        time_major=True,
        dtype=tf.float32)